In [ ]:
from typing import Dict, List, Tuple
from datamodel import *

class Trader:
    def run(self, state: TradingState) -> Dict[str, List[Order]]:
        """
        Only method required. It takes all buy and sell orders for all symbols as an input,
        and outputs a list of orders to be sent
        """
        # Initialize the method output dict as an empty dict
        result = {}

        # TODO: figure out why this sometimes breaks position limits
        MAX_PEARLS_BUY, MAX_PEARLS_SELL = self.get_max_orders(state.position, 20, "PEARLS")

        # Iterate over all the keys (the available products) contained in the order depths
        for product in state.order_depths.keys():
            orders: list[Order] = []

            if product == 'PEARLS':
                order_depth: OrderDepth = state.order_depths[product]
                acceptable_price = 10000

                if len(order_depth.sell_orders) > 0:
                    # Sort all the available sell orders by their price,
                    # and select only the sell order with the lowest price
                    best_ask = min(order_depth.sell_orders.keys())
                    best_ask_volume = min(MAX_PEARLS_BUY, order_depth.sell_orders[best_ask])
                    if best_ask < acceptable_price:
                        print("BUY", str(-best_ask_volume) + "x", best_ask)
                        orders.append(Order(product, best_ask, -best_ask_volume))

                if len(order_depth.buy_orders) != 0:
                    # Sort all the available buy orders by their price,
                    # and select only the buy order with the highest price
                    best_bid = max(order_depth.buy_orders.keys())
                    best_bid_volume = min(MAX_PEARLS_SELL, order_depth.buy_orders[best_bid])
                    if best_bid > acceptable_price:
                        print("SELL", str(best_bid_volume) + "x", best_bid)
                        orders.append(Order(product, best_bid, -best_bid_volume))

                result[product] = orders

        return result

    '''
    Returns MAX_BUY_ORDERS and MAX_SELL_ORDERS as a pair
    '''
    def get_max_orders(self,
                       position: Dict[Product, Position],
                       max_position: Position,
                       product: Product) -> Tuple[Position, Position]:
        MAX_BUY = max_position - position.get(product, 0)
        MAX_SELL = max_position + position.get(product, 0)
        return MAX_BUY, MAX_SELL

In [ ]:
from typing import Dict, List, Tuple
from datamodel import *

class Trader:
    def run(self, state: TradingState) -> Dict[str, List[Order]]:
        """
        Only method required. It takes all buy and sell orders for all symbols as an input,
        and outputs a list of orders to be sent
        """
        # Initialize the method output dict as an empty dict
        result = {}

        # TODO: figure out why this sometimes breaks position limits
        MAX_PEARLS_BUY, MAX_PEARLS_SELL = self.get_max_orders(state.position, 20, "PEARLS")

        # Iterate over all the keys (the available products) contained in the order depths
        for product in state.order_depths.keys():
            orders: list[Order] = []

            if product == 'PEARLS':
                order_depth: OrderDepth = state.order_depths[product]
                acceptable_price = 10000

                if len(order_depth.sell_orders) > 0:
                    # Sort all the available sell orders by their price,
                    # and select only the sell order with the lowest price
                    best_ask = min(order_depth.sell_orders.keys())
                    best_ask_volume = min(MAX_PEARLS_BUY, order_depth.sell_orders[best_ask])
                    if best_ask < acceptable_price:
                        print("BUY", str(-best_ask_volume) + "x", best_ask)
                        orders.append(Order(product, best_ask, -best_ask_volume))

                if len(order_depth.buy_orders) != 0:
                    # Sort all the available buy orders by their price,
                    # and select only the buy order with the highest price
                    best_bid = max(order_depth.buy_orders.keys())
                    best_bid_volume = min(MAX_PEARLS_SELL, order_depth.buy_orders[best_bid])
                    if best_bid > acceptable_price:
                        print("SELL", str(best_bid_volume) + "x", best_bid)
                        orders.append(Order(product, best_bid, -best_bid_volume))

                result[product] = orders

        return result

    '''
    Returns MAX_BUY_ORDERS and MAX_SELL_ORDERS as a pair
    '''
    def get_max_orders(self,
                       position: Dict[Product, Position],
                       max_position: Position,
                       product: Product) -> Tuple[Position, Position]:
        MAX_BUY = max_position - position.get(product, 0)
        MAX_SELL = max_position + position.get(product, 0)
        return MAX_BUY, MAX_SELL

In [1]:
#@title
# datamodel.py

import json
from typing import Dict, List
from json import JSONEncoder

Time = int
Symbol = str
Product = str
Position = int
UserId = str
Observation = int


class Listing:
    def __init__(self, symbol: Symbol, product: Product, denomination: Product):
        self.symbol = symbol
        self.product = product
        self.denomination = denomination


class Order:
    def __init__(self, symbol: Symbol, price: int, quantity: int) -> None:
        self.symbol = symbol
        self.price = price
        self.quantity = quantity

    def __str__(self) -> str:
        return "(" + self.symbol + ", " + str(self.price) + ", " + str(self.quantity) + ")"

    def __repr__(self) -> str:
        return "(" + self.symbol + ", " + str(self.price) + ", " + str(self.quantity) + ")"
    

class OrderDepth:
    def __init__(self):
        self.buy_orders: Dict[int, int] = {}
        self.sell_orders: Dict[int, int] = {}


class Trade:
    def __init__(self, symbol: Symbol, price: int, quantity: int, buyer: UserId = None, seller: UserId = None, timestamp: int = 0) -> None:
        self.symbol = symbol
        self.price: int = price
        self.quantity: int = quantity
        self.buyer = buyer
        self.seller = seller
        self.timestamp = timestamp

class TradingState(object):
    def __init__(self,
                 timestamp: Time,
                 listings: Dict[Symbol, Listing],
                 order_depths: Dict[Symbol, OrderDepth],
                 own_trades: Dict[Symbol, List[Trade]],
                 market_trades: Dict[Symbol, List[Trade]],
                 position: Dict[Product, Position],
                 observations: Dict[Product, Observation]):
        self.timestamp = timestamp
        self.listings = listings
        self.order_depths = order_depths
        self.own_trades = own_trades
        self.market_trades = market_trades
        self.position = position
        self.observations = observations
        
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True)
    
class ProsperityEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__